# 误差反向传播法

作者：杨岱川

时间：2019年12月

github：https://github.com/DrDavidS/basic_Machine_Learning

开源协议：[MIT](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/LICENSE)

参考文献：

- 《深度学习入门》，作者：斋藤康毅；
- 《深度学习》，作者：Ian Goodfellow 、Yoshua Bengio、Aaron Courville。
- [Keras overview](https://tensorflow.google.cn/guide/keras/overview)
- [PyTorch Tutorials](https://pytorch.org/tutorials/)
- [CS231n](http://cs231n.stanford.edu/)

## 本节目的

在[3.02 神经网络的训练](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/%E6%B7%B1%E5%BA%A6%E5%AD%A6%E4%B9%A0%E5%9F%BA%E7%A1%80/3.02%20%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C%E7%9A%84%E8%AE%AD%E7%BB%83.ipynb)中，我们简单介绍了神经网络是如何学习的，然后通过数值微分的方法计算了神经网络的损失函数 $L$ 关于权重参数 $W$ 的梯度，也就是 $\cfrac{\partial L}{\partial W}$。数值微分的方式简单，实现方便，但是微分操作在计算效率上很低。所以我们现在会学习一个能够高效计算梯度的方法 —— **反向传播(back propagation)算法**。

为了更精确和清楚地描述反向传播法算法，我们会采用基于**计算图（computational graph）**的形式。计算图描述会更加地直观，然后结合代码加深理解。

## 计算图

计算图将计算过程用图的形式表现出来，这里的图形就是数据结构图，通过多个节点和边表示。为了熟悉计算图，我们先用计算图解一下简单的，然后从这些简单问题出发，逐步深入，最终抵达误差反向传播法。

### 使用计算图求解

首先我们使用一个非常简单的例子作为开始。这个例子即使心算也能解答，这里的目的主要是让大家熟悉计算图。明白计算图的原理后，我们会在后面的复杂计算中看到它所发挥的巨大作用。

>首先给大家讲一讲经济常识，**价外税**的概念。增值税是一种价外税，价外税税款不包含在商品价格内的税，它是按照税收与价格的组成关系对税收进行的分类。实际上，为了方便起见，我们买的绝大多数商品的价格已经包含了增值税，以现行的增值税税率，水果的税率在13%。

>**假定**：
>
>这里我们假定，有这么一家超市，超市里面商品的标价都是**税前价格**。
>
>为了方便计算，我们再假定增值税的税率为10%，不过请记住，现在的大多数商品增值税税率是13%，少部分商品是9%。
>
> **问题1**：我在超市买了 2个 税前原价 100块 一个的苹果，苹果的增值税是 10% ，请计算我实际应该付多少钱？

计算图通过节点和箭头来表示计算过程。节点中是计算的**内容**，比如加法或者乘法之类，计算的**中间结果**写在箭头的上方。

![img](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/%E8%AE%A1%E7%AE%97%E5%9B%BE1.jpg?raw=true)

如上图所示，苹果的 100元 流动到了 “$\times2$” 节点，变成 200元，然后传递到下一个节点。然后这个 200元 流向 “$\times1.1$” 节点，变成 220元。因此，从这个计算图可以知道，我应该付 220元。

实际上 “$\times2$”和“$\times1.1$” 等节点作为一个运算整体用圆圈 $○$ 括起来了，不过我们也可以只用 $○$ 来表示乘法运算 $\times$。

如下图所示，我们可以将 “2” 和 “1.1” 分别作为变量 “苹果的个数” 和 “增值税” 标在 $○$ 外面：

![img](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/%E8%AE%A1%E7%AE%97%E5%9B%BE2.jpg?raw=true)

再看下一题：

>**问题2**：我在超市买了 2个 税前原价 100块 一个的苹果，以及 3个 税前原件 150块 一个的橘子，增值税是 10% ，请计算我实际应该付多少钱？

同上面的问题，我们还是用计算图来解决问题2，过程如下图：

![img](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/%E8%AE%A1%E7%AE%97%E5%9B%BE3.jpg?raw=true)

在这个问题中我们新增了加法节点 “$+$” ，用来计算苹果和橘子的合计金额。构建了计算图后，我们从左到右进行计算，就向电路中的电流流动，计算结果也从左边传递到了右边。

到达最右边的计算结果以后，计算过程就结束了，从图中可以知道，问题2的答案是715元。

综上两个简单例子，我们总结一下计算图解题的流程情况：

>1. 构建计算图
>
>2. 在计算图上，从左向右进行计算。

这里第2步 “从左向右进行计算” 正是我们之前学习过的**前向传播（forward propagation）**。前向传播也叫正向传播，是从计算图出发点到结束点的传播。既然有前向传播，自然我们会想到有反向的情况，从图上就是从右到左。这种传播就是**反向传播（backward propagation）**。反向传播会在导数计算中发挥重要作用。

### 局部计算

计算图的特点就是可以通过传递 **“局部计算”** 获得最终结果。“局部” 的意思就是 “与自己相关的某个小范围”，所以局部计算就是指，无论全局发了什么，都能只根据与自己相关的信息输出接下来的结果。

![img](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/%E8%AE%A1%E7%AE%97%E5%9B%BE4-%E6%9B%B4%E6%96%B0.jpg?raw=true)

如上图所示，假设（经过了复杂的计算）我们购买了很多其他的东西，总共花了4000元。这里的重点在于，节点处的计算都是局部的计算，这表明，例如苹果和其他很多东西的求和运算 （$4000+200\to4200$）并不关心 4000 这个数字是怎么计算而来的，只要简单相加就行了。换言之，各个节点处只需要进行与自己有关的计算，不用考虑全局。

综上，计算图可以集中精力于局部计算。无论全局的计算有多么复杂，各个步骤要做的就是对象节点的局部计算。虽然局部计算简单，但是通过传播它的计算结果，可以获得全局的复杂计算的结果，就像汽车厂里面的流水制造线一样。

### 为何使用计算图

前面我们用计算图举了两个很简单的例子，解决了两个问题。那么计算图的优点到底有什么？

一是刚刚提到的局部计算，无论全局是多么复杂，都可以通过局部计算使得各个节点致力于简单的计算，从而简化问题。

二是利用计算图可以保存中间的计算结果，比如刚刚我们计算到苹果和橘子等的分别购买的金额。

三则是最重要的原因，可以**通过反向传播高效计算导数**。

回到最开始的**问题1**，我们一开始计算的是，购买了 2 个苹果加上增值税后最终需要支付的金额。如果我们想要知道，有一天苹果涨价了，那么会在多大程度上影响最终支付的金额呢？

这相当于求 “支付金额关于苹果价格的导数”。假设苹果的价格是 $x$，支付金额为 $L$，则相当于求 $\cfrac{\partial L}{\partial x}$，这个导数的值就表示当苹果价格上涨的时候，支付金额会变多少。

>这里的支付金额函数 $L$有 3 个变量，一个是苹果价格，设为 $x$，另一个是税率，设为 $t$，苹果的数量设为 $n$，故有
>
>$$\large L = n\times x \times t$$
>
>所以，如果想求苹果价格变化导致最终支付结果变化的幅度，求的是 $L$ 关于 $x$ 的偏导数。

如前所述， “支付金额关于苹果价格的导数”的值可以通过计算图的反向传播求出来，至于反向传播具体怎么求，稍后就会介绍。先看看结果：

![img](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/%E5%8F%8D%E5%90%91%E4%BC%A0%E6%92%AD0.jpg?raw=true)

如上图，反向传播以橙色箭头表示。反向传播传递“局部导数”，将导数的值写在箭头下方。

在这里，反向传播从右到左传递导数的值（$1\to 1.1\to 2.2$）。从这个结果可以知道 “支付金额关于苹果价格的导数”的值为 2.2。换句话说，苹果价格每上涨 1 元，最终支付的金额会多 2.2 元。

>再严格一点，如果苹果的价格增加一个微小值 $\Delta h$，那么最终支付的金额就会增加 $2.2\Delta h$。

这里我们求了 “支付金额关于苹果价格的导数”，同理我们也可以求 “支付金额关于苹果数量的导数”、“支付金额关于增值税的导数”，而且计算中途得到的导数结果（及中间传递的导数）可以被共享，从而可以高效地计算多个导数。

总结一下，计算图的优点就是可以通过正向传播和反向传播高效地计算各个变量的导数。

## 链式法则

前面的计算图主要讲的就是结果的前向传播，和我们平时接触的计算过程相符合，所以感觉上很自然。而反向传播将局部导数向正方向的反方向（从右到左）传递，可能会让人觉得困惑。

其实，传递这个局部导数的原理，是基于**链式法则（chain rule）**的，接下来我们会介绍链式法则，然后阐明其是如何对应计算图上的反向传播。

### 计算图的反向传播

首先看一个例子，假设存在一个 $y=f(x)$ 的计算，其反向传播如图：

![img](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/%E5%8F%8D%E5%90%91%E4%BC%A0%E6%92%AD1.jpg?raw=true)

如上图所示，反向传播的计算顺序是，将信号 $E$ 乘以节点的局部导数 $\cfrac{\partial y}{\partial x}$，然后将结果传递给下一个节点。这里所说的局部导数是指正向传播中 $y=f(x)$ 的导数，也就是 $y$ 关于 $x$ 的导数 $\cfrac{\partial y}{\partial x}$。

比如，假设 $y=f(x)=x^2$，则局部导数 $\cfrac{\partial y}{\partial x} = 2x$。把这个局部导数乘以上游传过来的值（这里是 $E$ ），然后传递给前面的节点。

这就是反向传播的过程。通过这样的计算，我们可以高效地求出导数的值，这是反向传播的要点。至于具体实现，我们需要从链式法则说起。

### 链式法则简介

首先，我们回顾一下**复合函数**。

复合函数是由多个函数构成的函数，比如 $z={(x+y)}^2$ 是由下面两个式子构成的：

$$
\large 
\begin{equation}\begin{split} 
z&=t^2\\
t&=x+y
\end{split} \end{equation}
 \tag{5.1}$$

链式法则是关于复合函数导数的性质，定义：

- 如果某个函数由复合函数表示，则该复合函数的导数可以用构成复合函数的各个函数的导数的乘积表示。

实际上这是一个非常简单的性质，以上面的函数为例，$\cfrac{\partial z}{\partial x}$（$z$ 关于 $x$ 的导数）可以用 $\cfrac{\partial z}{\partial t}$ （$z$ 关于 $t$ 的导数）和 $\cfrac{\partial t}{\partial x}$ （$t$ 关于 $x$ 的导数）的乘积来表示。

用数学式表示，就是：

$$\large \frac{\partial z}{\partial x}=\frac{\partial z}{\partial t}\frac{\partial t}{\partial x} \tag{5.2}$$

然后其中两个 $\partial t$ 相互消除，就成了

$$\large \require{cancel} \frac{\partial z}{\partial x}=\frac{\partial z}{\cancel {\partial t}}\frac{\cancel{\partial t}}{\partial x}$$

现在我们使用链式法则来求式 5.2 的 $\cfrac{\partial z}{\partial x}$，首先我们要求偏导：

$$
\large 
\begin{equation}\begin{split} 
\frac{\partial z}{\partial t} &= 2t\\
\frac{\partial t}{\partial x} &= 1
\end{split} \end{equation}
 \tag{5.3}$$

如式 5.3 所示，$\cfrac{\partial z}{\partial t}$ 等于 $2t$，$\cfrac{\partial t}{\partial x}$ 等于 $1$。这是基于导数公式的解析解。然后最后要计算的$\cfrac{\partial z}{\partial x}$ 可以根据链式法则，计算导数乘积而得：

$$\large \frac{\partial z}{\partial x}=\frac{\partial z}{\partial t}\frac{\partial t}{\partial x}=2t\cdot1=2(x+y) \tag{5.4}$$

### 链式法则和计算图

现在我们尝试将式 5.4 的链式法则的计算用计算图来表示，其中平方运算我们采用 “$**2$” 代替：

![img](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/%E5%8F%8D%E5%90%91%E4%BC%A0%E6%92%AD2.1.jpg?raw=true)

如图所示，计算图的反向传播从右到传播信号。反向传播的顺序是，先将节点的输入信号乘以节点的局部导数（偏导数），然后再传递给下一个节点。

比如，反向传播的时候，“$**2$” 节点的输入是 $\cfrac{\partial z}{\partial z}$，将其乘以局部导数 $\cfrac{\partial z}{\partial t}$（因为正向传播时候，输入是 $t$，输出是 $z$，所以这个节点的局部导数是 $\cfrac{\partial z}{\partial t}$ ），然后传递给下一个节点。

另外，上图中反向传播最开始的信号  $\cfrac{\partial z}{\partial z}$ 在前面的数学式中是没有出现的，因为  $\cfrac{\partial z}{\partial z}=1$，所以省略了。

图中还需要注意的是最左边的反向传播的结果。根据链式法则，$\cfrac{\partial z}{\partial z}\cfrac{\partial z}{\partial t}\cfrac{\partial t}{\partial x}=\cfrac{\partial z}{\partial t}\cfrac{\partial t}{\partial x}=\cfrac{\partial z}{\partial x}$ 成立，对应 “$z$ 关于 $x$ 的导数”。也就是说，反向传播是基于链式法则的。 

把式 5.3 的结果代入上图中，结果如下图所示， $\cfrac{\partial z}{\partial x}$ 的结果为 $2(x+y)$。

![img](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/%E9%93%BE%E5%BC%8F%E6%B3%95%E5%88%992.png?raw=true)

## 反向传播

之前我们介绍了计算图的反向传播是基于链式法则成立的，现在我们将以 “$+$” 和 “$\times$” 等运算为例，介绍反向传播的结构。

### 加法节点的反向传播

首先我们来看看加法节点的反向传播。

以 $z=x+y$ 为对象，观察其反向传播。 $z=x+y$ 的导数可由下式计算：

$$
\large 
\begin{equation}\begin{split} 
\frac{\partial z}{\partial x}&=1\\[2ex]
\frac{\partial z}{\partial y}&=1
\end{split} \end{equation}
 \tag{5.5}$$
 
如式 5.5 所示，$\cfrac{\partial z}{\partial x}$ 和 $\cfrac{\partial z}{\partial y}$ 同时都等于1，因此用计算图表示，就如下图所示：

![img](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/%E5%8A%A0%E6%B3%95%E7%9A%84%E5%8F%8D%E5%90%91%E4%BC%A0%E6%92%AD.png?raw=true)

在上图中，反向传播将从上游传过来的导数（这里是 $\cfrac{\partial L}{\partial z}$）乘以 $1$，然后传向下游。也就是说，因为加法节点的反向传播只乘以 $1$，所以输入的值会原封不动地流向下一个节点。

至于本例中，我们把上游传过来的导数的值设为 $\cfrac{\partial L}{\partial z}$，这是因为，我们假定了一个最终输出值为 $L$ 的大型计算图，如下：

![img](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/%E5%8A%A0%E6%B3%95%E7%9A%84%E5%8F%8D%E5%90%91%E4%BC%A0%E6%92%AD2.png?raw=true)

$z=x+y$ 的计算位于这个大型计算图的某个地方，从上游会传来 $\cfrac{\partial L}{\partial z}$ 的值，并向下游传递 $\cfrac{\partial L}{\partial x}$ 和 $\cfrac{\partial L}{\partial y}$。

举个例子，假设有“$10+5=15$”这样一个计算，且反向传播时，会从上游传来值 $1.3$（注意这个 $1.3$ 是我们假设的上游值）。用计算图表示，如下图

![img](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/%E5%8A%A0%E6%B3%95%E7%9A%84%E5%8F%8D%E5%90%91%E4%BC%A0%E6%92%AD3.png?raw=true)

因为加法结点的反向传播只是将信号输出到下一个结点，所以如上图，反向传播将 $1.3$ 向下一个结点传递。

### 乘法节点的反向传播

加法之后是乘法，我们看看乘法节点的反向传播。我们考虑 $z=xy$。这个式子用导数表示就是：

$$
\large 
\begin{equation}\begin{split} 
\frac{\partial z}{\partial x}&=y\\[2ex]
\frac{\partial z}{\partial y}&=x
\end{split} \end{equation}
 \tag{5.6}$$

根据式 5.6，我们可以画出计算图。

![img](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/%E4%B9%98%E6%B3%95%E7%9A%84%E5%8F%8D%E5%90%91%E4%BC%A0%E6%92%AD1.png?raw=true)

乘法的反向传播会将上游的值乘以正向传播时的输入信号的“翻转值”后传递给下游。

翻转值表示一种翻转关系，如图所示，正向传播的信号是 $x$ 的话，反向传播时候就是 $y$；正向传播时候信号是 $y$ 的话，反向传播时就是 $x$。

现在来看一个具体例子，比如有 “$10\times 5 = 50$” 这样一个计算，反向传播时候，上游传来一个值 $1.3$。用计算图表示为：

![img](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/%E4%B9%98%E6%B3%95%E7%9A%84%E5%8F%8D%E5%90%91%E4%BC%A0%E6%92%AD2.png?raw=true)

由于乘法的反向传播会乘以输入信号的翻转值，所以各自按照 $1.3\times5=6.5$、$1.3\times10=13$ 计算。另外，加法的反向传播并不需要正向传播的输入信号。但是，乘法的反向传播需要正向传播的输入信号值（这里是 5 和 10），因此，实现乘法节点的反向传播时，需要保存正向传播的输入信号。

## 简单层的实现

回顾一下之前购买苹果的例子，我们将用 Python 来实现这个例子。这里，我们把要实现的计算图乘法节点称为“乘法层（MulLayer）”，加法节点称为“加法层（AddLayer）”。

### 乘法层的实现

每个层的实现都应该有两个共通的方法：**forward()** 和 **backward()**。forward() 对应前向传播，而 backward() 对应反向传播。

现在实现乘法层。乘法层作为 MulLayer 类，实现过程如下：

In [1]:
class MulLayer:
    """乘法层"""
    def __init__(self):
        """初始化"""
        self.x = None
        self.y = None
    
    def forward(self, x, y):
        """前向传播"""
        self.x = x
        self.y = y
        out = x * y
        return out

    def backward(self, dout):
        """反向传播"""
        dx = dout * self.y  # 翻转x和y
        dy = dout * self.x
        return dx, dy

在上述代码中，`__init__()` 会初始化实例变量 $x$ 和 $y$，它们用于保存正向传播时的输入值。`forward()` 接收 $x$ 和 $y$ 两个参数，将它们相乘后输出。`backward()` 将从上游传过来的导数（dout）乘以前向传播的翻转值，然后传给下游。

现在我们可以使用 `MulLayer` 来实现前面购买苹果的例子（两个苹果和增值税）。

![img](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/%E4%B9%98%E6%B3%95%E7%9A%84%E5%8F%8D%E5%90%91%E4%BC%A0%E6%92%AD3.png?raw=true)

使用这个乘法层，上图的正向传播可以如下实现：

In [2]:
apple = 100
apple_num = 2
tax = 1.1

In [3]:
# layer
mul_apple_layer = MulLayer()
mul_tax_layer = MulLayer()

# forward
apple_price = mul_apple_layer.forward(apple, apple_num)
price = mul_tax_layer.forward(apple_price, tax)

print("price:", int(price))  # 220

price: 220


此外，关于各个变量的导数可以由 `backward()` 求出：

In [4]:
# backward
dprice = 1
dapple_price, dtax = mul_tax_layer.backward(dprice)
dapple, dapple_num = mul_apple_layer.backward(dapple_price)

print("price:", int(price))
print("dApple:", dapple)
print("dApple_num:", int(dapple_num))
print("dTax:", dtax)

price: 220
dApple: 2.2
dApple_num: 110
dTax: 200


可以看出，运行结果和上面的图中的结果是一样的。

>注意，`backward()` 的参数中需要输入 “关于正向传播时的输出变量的导数”。比如，`mul_apple_layer` 乘法层在正向传播时会输出 apple_price，在反向传播时，则会将 `apple_price` 的导数 `dapple_price` 设为参数。

### 加法层的实现

现在我们来看看加法层是如何实现的，首先看看代码：

In [5]:
class AddLayer:
    """加法层"""
    def __init__(self):
        """
        初始化
        
        这里的 pass 指的是，加法层不需要保存正向传播的输入信号
        """
        pass
    
    def forward(self, x, y):
        """前向传播"""
        out = x + y
        return out
    
    def backward(self, dout):
        """反向传播"""
        dx = dout * 1
        dy = dout * 1
        return dx, dy

加法层不需要特意初始化，所以 `__init__()` 中什么也不运行。

加法层的 `forward()` 接收 $x$ 和 $y$ 两个参数，将他们相加后输出。而 `backward()`，我们在前面已经讨论过了，它将上游传来的导数原封不动地传给下游。

有兴趣的同学可以参考《深度学习入门》第 5.4.2 节中的代码实现简单的加法和乘法层的组合，代码参考[这里](https://github.com/oreilly-japan/deep-learning-from-scratch/blob/master/ch05/buy_apple_orange.py)。

## 激活函数层的实现

除了简单的加法和乘法以外，在[3.1 神经网络与前向传播](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/%E6%B7%B1%E5%BA%A6%E5%AD%A6%E4%B9%A0%E5%9F%BA%E7%A1%80/3.01%20%E7%A5%9E%E7%BB%8F%E7%BD%91%E7%BB%9C%E4%B8%8E%E5%89%8D%E5%90%91%E4%BC%A0%E6%92%AD.ipynb) 中我们还学习过**激活函数**，所以现在我们要实现激活函数的反向传播。

应用计算图的思路，和之前对加法层、乘法层的实现，我们也把神经网络的其他部分实现为一个**类（class）**，这里首先我们来试试实现激活函数的 **ReLU** 层和 **Sigmoid** 层。

### ReLU层

回忆一下激活函数 ReLU(Rectified Linear Unit) 的表示形式：

$$
\large
y =
\begin{cases}
x, & (x \gt 0)  \\
0, & (x \le 0)
\end{cases} \tag{5.7}
$$

通过式 5.7 我们可以求出 $y$ 关于 $x$ 的导数，如式 5.8：

$$
\large
\frac{\partial y}{\partial x} =
\begin{cases}
1, & (x \gt 0)  \\
0, & (x \le 0)
\end{cases} \tag{5.8}
$$

在式 5.8 中，如果前向传播时的输入 $x$ 大于 0 ，则反向传播会将上游的值原封不动地传给下游。反过来，如果前向传播时的 $x$ 小于等于 0，则反向传播中传给下游的信号将停在此处。

用计算图的话，如下：

![img](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/ReLU%E7%9A%84%E5%8F%8D%E5%90%91%E4%BC%A0%E6%92%AD.png?raw=true)

现在我们来实现 ReLU 层。在神经网络的层的实现中，一般假定 `forward()` 和 `backward()` 的参数是 NumPy 数组。

In [6]:
class Relu:
    """ReLU 激活函数"""
    def __init__(self):
        self.mask = None
        
    def forward(self, x):
        self.mask = (x <= 0)
        out = x.copy()
        out[self.mask] = 0
        return out

    def backward(self, dout):
        dout[self.mask] = 0
        dx = dout
        return dx

其中，`mask` 的作用如下:

In [7]:
x = -1
mask = (x <= 0)
print(mask)

True


In [8]:
import numpy as np

x = np.array([[1, 2, 3], [-1, -2, -3]])
mask = (x <= 0)
print(mask)

[[False False False]
 [ True  True  True]]


`Relu` 类有实例变量 `mask`。这个变量 `mask` 是由 `True` \\ `False` 构成的 NumPy 数组，它会把正向传播时的输入 `x` 的元素中小于等于 0 的地方保存为 `True`，其他地方保存为 `False`

如上面的图所示，如果正向传播时的输入值小于等于 0 ，则反向传播的值为 0。因此，反向传播中会使用正向传播时保存的 mask，将从上游传来的 dout 的 mask 中的元素为 `True` 的地方设为 0。

### Sigmoid层

接下来我们实现 sigmoid 函数。 回顾一下 sigmoid 函数：

$$\large y=\frac{1}{1+e^{-x}}\tag{5.9}$$

用计算图表示式 5.9，如下图：

![sigmoid反向01update2](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/sigmoid%E5%8F%8D%E5%90%9101update2.png?raw=true)

上图中，除了 “$\times$” 和 “$+$” 节点外，还出现了新的 “$\exp$” 和 “$/$”节点。

“$\exp$” 节点会进行 $y=\exp(x)$ 的运算，即 $y=e^{-x}$，而 “$/$” 节点会进行 $y=\cfrac{1}{x}$ 的计算。

sigmoid的前向传播图由较为复杂的局部计算构成，因此我们依次来看看反向传播的流程。

#### 步骤1

“$/$” 节点表示 $y=\cfrac{1}{x}$ ，它的导数可以解析地表示为：

$$\large \frac{\partial y}{\partial x}=-\frac{1}{x^2}=-y^2 \tag{5.10}$$

根据式 5.10 ，反向传播时，会将上游的值乘以 $-y^2$ （即正向传播的输出的平方乘以 $-1$）后，再传给下游。计算图如下：

![sigmoid反向02update1](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/sigmoid%E5%8F%8D%E5%90%9102update1.png?raw=true)

#### 步骤2

“$+$” 节点将上游的值原封不动传给下游。如下图所示：

![sigmoid反向03update1](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/sigmoid%E5%8F%8D%E5%90%9103update1.png?raw=true)

#### 步骤3

“$\exp$” 节点表示 $y=e^{x}$，它的导数表示为：

$$\large \frac{\partial y}{\partial x}=e^x\tag{5.11}$$

在计算图中，上游的值乘以正向传播时的输出（本例中是 $e^{-x}$）后，再传给下游：

![sigmoid反向04update1](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/sigmoid%E5%8F%8D%E5%90%9104update1.png?raw=true)

> 注意，这里的 $e^x$ 求导之后是它本身，所以反向传播时候就是上游的导数值（比如 $f'(x)$）乘以正向的输出 $e^x$。

#### 步骤4

“$\times$” 节点将正向传播时的值翻转后做乘法运算。因此这里要乘以 $-1$：

![sigmoid反向05update1](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/sigmoid%E5%8F%8D%E5%90%9105update1.png?raw=true)

根据上述描述，步骤 4 中的计算图就可以进行 Sigmoid 层的反向传播。从图中可以知道，反向传播的输出为 $\cfrac{\partial L}{\partial y}y^2e^{-x}$，这个值会传播给下游的节点。

注意，$\cfrac{\partial L}{\partial y}y^2e^{-x}$ 这个值只根据正向传播时的输入 $x$ 和输出 $y$ 就可以算出来，因此我们可以把 “sigmoid” 节点集约化一下：

![sigmoid06](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/sigmoid%E5%8F%8D%E5%90%9106.png?raw=true)

上图就是简洁版本的计算图，我们可以省略中间的反向传播过程，计算效率更高。

> 通过对节点的“集约化”，可以不用在意 Sigmoid 层中琐碎的细节，只需要专注它的输入和输出，这一点很重要。

此外，$\cfrac{\partial L}{\partial y}y^2e^{-x}$ 还可以进一步整理：

$$
\large
\begin{equation}\begin{split} 
\frac{\partial L}{\partial y}y^2e^{-x}&=\frac{\partial L}{\partial y}\frac{1}{\left(1+e^{-x}\right)^2}e^{-x} \\ 
&=\frac{\partial L}{\partial y}\frac{1}{1+e^{-x}}\frac{e^{-x}}{1+e^{-x}}\\ 
&=\frac{\partial L}{\partial y}y(1-y)
\end{split}\end{equation}\tag{5.12}
$$

现在，由式 5.12 可得，Sigmoid 层的反向传播只根据正向传播的输出 $y$ 就能计算出来：

![sigmoid07](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/sigmoid%E5%8F%8D%E5%90%9107.png?raw=true)

现在我们使用 Python 代码对 Sigmoid 层进行实现：

In [9]:
class Sigmoid:
    """Sigmoid层"""
    def __init__(self):
        """初始化"""
        self.out = None
    
    def forward(self, x):
        """前向传播"""
        out = 1 / (1 + np.exp(-x))
        self.out = out
        return out

    def backward(self, dout):
        """
        反向传播
        
        Keyword arguments:
        dout -- 上游的导数
        """
        dx = dout * (1.0 - self.out) * self.out
        return dx

## Affine / Softmax 层的实现

### Affine 层

Affine层就是全连接层，又叫仿射层。

神经网络的正向传播中，为了计算加权信号的总和，使用了矩阵的乘积运算（NumPy 中的 `np.dot()` 函数）。

在之前神经网络的前向传播中，我们曾经实现过最简单的部分，回顾一下：

In [10]:
import numpy as np

X = np.random.rand(2)     # 输入
# X = np.asmatrix(X)
W = np.random.rand(2, 3)  # 权重
# W = np.asmatrix(W)
B = np.random.rand(3)     # 偏置
# B = np.asmatrix(B)

In [11]:
print(f"X：{X} ,\nX.shape：{X.shape}\n")
print(f"W：{W} ,\nW.shape：{W.shape}\n")
print(f"B：{B} ,\nB.shape：{B.shape}\n")

X：[0.95099499 0.86236951] ,
X.shape：(2,)

W：[[0.27628981 0.77139771 0.24660224]
 [0.32572545 0.7945617  0.13803012]] ,
W.shape：(2, 3)

B：[0.62884398 0.17308075 0.38642165] ,
B.shape：(3,)



In [12]:
Y = np.dot(X, W) + B
print(Y)

[1.1724899  1.59188189 0.7399721 ]


这里，$\textbf{X}$、$\textbf{W}$、$\textbf{B}$ 分别是形状为 $(2,)$、$(2,3)$、$(3,)$ 的多维数组。

>如果觉得逗号后面空一格难受，可以使用 `np.asmatrix()` 把多维数组转换为矩阵。

所以神经元的加权和可以用

```python
Y = np.dot(X, W) + B
```

算出来。然后 $\textbf{Y}$ 经过激活函数的转换后，传递给下一层。这就是神经网络正向传播的流程。

现在将这里进行的求矩阵的乘积与偏置的和的运算用计算图表示出来。将乘积运算用 “dot” 节点表示的话，则 `np.dot(X, W) + B` 的运算可以用下图所示计算图表示出来。

![affine1](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/affine%E5%B1%82%E7%9A%84%E5%AE%9E%E7%8E%B01-update1.png?raw=true)

上图是比较简单的计算图，要注意的是 $\textbf{X}$、$\textbf{W}$、$\textbf{B}$ 是**矩阵**（多维数组）。而之前我们见到的计算图中流动的都是标量。

现在我们来考虑一下如何实现上图的反向传播。以矩阵为对象的反向传播，按矩阵的各个元素进行计算时，步骤与以标量为对象的计算图一致。这里省略了推导（可以参考后面的计算图的反向传播），直接给出结果如下：

$$
\large 
\begin{equation}\begin{split} 
\frac{\partial L}{\partial \textbf{X}}&=\frac{\partial L}{\partial \textbf{Y}}\cdot {\textbf{W}}^{\rm T}\\[2ex]
\frac{\partial L}{\partial \textbf{W}}&={\textbf{X}}^{\rm T} \cdot \frac{\partial L}{\partial \textbf{Y}}
\end{split} \end{equation}
 \tag{5.13}$$
 
其中 ${\textbf{W}}^{\rm T}$ 是 ${\textbf{W}}$ 的转置矩阵。

实际上我们可以试着画出计算图的反向传播过程：

![affine层的实现2-update3](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/affine%E5%B1%82%E7%9A%84%E5%AE%9E%E7%8E%B02-update3.png?raw=true)

我们看看上面计算图中各个变量的形状。注意，$\textbf{X}$ 和 $\cfrac{\partial L}{\partial \textbf{X}}$ 形状相同，$\textbf{W}$ 和 $\cfrac{\partial L}{\partial \textbf{W}}$ 形状相同。参考如下：

$$
\large 
\begin{equation}\begin{split} 
\textbf{X} &=(x_0,x_1,\cdots,x_n)\\[2ex]
\frac{\partial L}{\partial \textbf{X}} &=\left( \frac{\partial L}{\partial x_0},\frac{\partial L}{\partial x_1},\cdots,\frac{\partial L}{\partial x_n} \right)
\end{split} \end{equation}
 \tag{5.15}$$

为什么要注意矩阵形状呢？

因为矩阵的乘积运算要求对应元素个数保持一致，通过确认一致性，可以导出式 5.13。 比如 $\cfrac{\partial L}{\partial \textbf{Y}}$ 的形状是 $(3,)$ ，${\textbf{W}}$ 的形状是 $(2,3)$ 时，思考 $\cfrac{\partial L}{\partial \textbf{Y}}$ 和 ${\textbf{W}}^{\rm T}$ 的乘积，使得 $\cfrac{\partial L}{\partial \textbf{X}}$ 的形状为 $(2,)$。

这样一来，自然而然推导出式 5.13。

### 批版本的Affine层

前面介绍的 Affine 层的输入 $X$ 是以单个数据为对象的。实际上我们的数据传入通常是一批一批（batch）传入，现在我们考虑 $N$ 个数据一起进行正向传播的情况，也就是批版本的 Affine 层。

批版本的 Affine 层计算图如下：

![affine层的BATCH实现update1](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/affine%E5%B1%82%E7%9A%84BATCH%E5%AE%9E%E7%8E%B0update1.png?raw=true)

和刚刚不同的是，现在 $\textbf{X}$ 的输入形状是 $(N,2)$。之后和前面一样，在计算图上进行矩阵计算。反向传播时，注意矩阵的形状，就能和前面一样推导出 $\cfrac{\partial L}{\partial \textbf{X}}$ 和 $\cfrac{\partial L}{\partial \textbf{W}}$。

加上**偏置**时候要注意！前向传播时，偏置是被加到了 $\textbf{X}\cdot\textbf{W}$ 的每一个数据上（可以回顾一下前向传播的内容）。因此反向传播时，各个数据的反向传播的值需要汇总为偏置的元素。

参考代码:

In [13]:
dY = np.matrix([[1, 2, 3], [4, 5, 6]])
print(dY)

[[1 2 3]
 [4 5 6]]


In [14]:
# 加起来
dB = np.sum(dY, axis=0)
print(dB)

[[5 7 9]]


这个例子中，假定数据有 2 个（$N=2$）。偏置的反向传播会对这 2 个数据的导数按元素进行求和。因此，这里使用了 `np.sum()` 对第 0 轴（以数据为单位的轴，`axis=0`）方向上的元素进行求和。

综上所述，Affine 的实现如下：

In [15]:
class Affine:
    """仿射层"""
    def __init__(self, W, b):
        """初始化"""
        self.W = W
        self.b = b
        self.x = None
        self.dw = None
        self.db = None
    
    def forward(self, x):
        """前向传播"""
        self.x = x
        out = np.dot(x, self.W) + self.b
        return out
    
    def backward(self, dout):
        dx = np.dot(dout, self.W.T)
        self.dW = np.dot(self.x.T, dout)
        self.db = np.sum(dout, axis=0)
        return dx

### Softmax-with-Loss 层

最后简单介绍一下输出层的 softmax 函数。我们在 [3.02 神经网络的训练](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/03深度学习基础/3.02%20神经网络的训练.ipynb) 中学习了 softmax 函数，知道它会将输入值正规化之后再输出，而每一类输出所加起来的总和为 1。

softmax 层一般包含了作为损失函数的交叉熵误差（cross entropy error），所以称为“Softmax-with-Loss层”。

>Softmax-with-loss 层较为复杂，其推导过程可以参考 《深度学习入门》附录A 或者网上的一些文章，比如[神经网络多分类中softmax+cross-entropy的前向传播和反向传播过程](https://zhuanlan.zhihu.com/p/86184547)。

这里给出一个简化版本的计算图：

![Softmax-with-Loss 层](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/back_up_images/softmax-with-loss.png?raw=true)

如上图所示，softmax 函数记为 Softmax 层，交叉熵误差集为 Cross Entropy Error 层。这里假设进行 3 分类，从前面的层接收 3 个输入。

Softmax 层将输入 $(a_1,a_2,a_3)$ 正规化，输出 $(y_1,y_2,y_3)$。 Cross Entropy Error 层接收 Softmax 的输出 $(y_1,y_2,y_3)$ 和监督标签 $(t_1,t_2,t_3)$，从这些数据中输出损失 $L$。

这里要注意的是反向传播的结果。Softmax 层的反向传播得到了 $(y_1-t_1,y_2-t_2,y_3-t_3)$ 这样的整齐漂亮结果。由于 $(y_1,y_2,y_3)$ 是 Softmax 层的输出，$(t_1,t_2,t_3)$ 是监督数据，所以 $(y_1-t_1,y_2-t_2,y_3-t_3)$ 是 Softmax 层的输出和监督标签的差分。神经网络的反向传播会把这个差分表示的误差传递给前面的层，这是神经网络学习中的重要性质。

神经网络学习的目的就是通过调整权重参数，使神经网络的输出（Softmax的输出）接近监督标签。因此，必须将神经网络的输出与监督标签的误差高效地传递给前面的层。刚刚的 $(y_1-t_1,y_2-t_2,y_3-t_3)$ 正是 Softmax 层与监督标签的差，直接了当地表示了当前神经网络地输出与监督标签的误差。

现在来进行 Softmax-with-Loss 层的实现，过程如下：

In [16]:
def softmax(a):
    """定义 softmax 函数"""
    exp_a = np.exp(a)
    sum_exp_a = np.sum(exp_a)
    y = exp_a / sum_exp_a
    return y

def cross_entropy_error(y, t):
    """定义交叉熵损失函数"""
    delta = 1e-7
    return -np.sum(t * np.log(y + delta))

class SoftmaxWithLoss:
    """SoftmaxWithLoss"""
    def __init__(self, W, b):
        """初始化"""
        self.loss = None  # 损失
        self.y = None     # softmax的输出
        self.t = None     # 监督数据（one hot形式）
    
    def forward(self, x):
        """前向传播"""
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)
        return self.loss
    
    def backward(self, dout):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        return dx

其中，`softmax()` 和 `cross_entropy_error()` 函数都是 [3.02 神经网络的训练](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/03深度学习基础/3.02%20神经网络的训练.ipynb) 中照搬过来的。所以这里的实现很简单。

## 误差反向传播的实现

### 神经网络学习的全貌图

我们在这里回顾一下神经网络学习的全貌图。神经网络学习的步骤如下所示：

- **前提**

    神经网络中有合适的权重和偏置，调整权重和偏置以拟合训练数据的过程叫做学习。神经网络的学习分为以下四个步骤。
    
    
- **步骤一（mini-batch）**

    从训练数据中随机选择一部分数据，大小为 batch-size。
    
    
- **步骤二（计算梯度）**

    计算损失函数关于各个可训练参数（权重和偏置）的梯度。
    
    
- **步骤三（更新参数）**

    将可训练参数沿着梯度方向进行微小的更新。


- **步骤四（重复）**

    重复步骤一、步骤二、步骤三，直到一定次数，或者损失不再下降为止。
    
之前介绍过的误差反向传播法会在步骤2中出现。在 [3.02 神经网络的训练](https://github.com/DrDavidS/basic_Machine_Learning/blob/master/03深度学习基础/3.02%20神经网络的训练.ipynb) 中，我们利用数值微分的方法求得这个梯度。

数值微分实现虽然简单，但是计算耗费时间非常多，但是误差反向传播法可以高效快速地计算梯度，因为刚刚我们都是通过计算图求的解析解。

### 误差反向传播法的梯度确认

到目前为止，我们介绍了两种求梯度的方法，一种是基于**数值微分**的方法，另一种是解析性地求解数学式的方法。后一种方法通过使用**误差反向传播法**，即使存在大量的参数，也可以高效地计算梯度。因此在实际编程中，我们通常使用误差反向传播法。

数值微分的计算很耗费时间，而且如果有误差反向传播法的实现的话，就没有必要使用数值微分实现了。那么数值微分有什么用呢？

实际上，在确认误差反向传播法的实现是否正确时，需要使用数值微分。

数值微分的优点是实现简单，因此，在一般情况下不太容易出错。而误差反向传播法的实现很复杂，在编程实现时相对更容易出错，所以我们经常会比较数值微分和误差反向传播法的结果，以确认误差反向传播法的实现是否正确。

确认数值微分所求出的梯度的结果和误差反向传播法求出的的结果是否一致（非常非常接近，不一定完全一致）的操作叫做 **梯度确认（gradient check）**，这里只要求明白其涵义，而具体实现可以参考[gradient_check.py](https://github.com/oreilly-japan/deep-learning-from-scratch/blob/master/ch05/gradient_check.py)。

## 小结

这次学习中我们介绍了将计算过程可视化的**计算图**，并且使用计算图，介绍了神经网络中的**误差反向传播法**，然后以层（layer）为单位实现了神经网络中的处理。

我们学过的层有：

- ReLU
- Softmax-with-Loss
- Affine
- Softmax

等，并且在其中通过计算图解释了其**前向传播**和**反向传播**的原理，实现了 `forward()` 和 `backward()` 方法。

通过数据的前向传播和反向传播，我们可以高效地计算权重参数地梯度。然后通过对层的模块化实现，我们可以自由地组装层，轻松构建出自己想要的网络结构。